## Modbus 的相關說明

- 首先可以來看看維基怎麼說： https://zh.wikipedia.org/wiki/Modbus
- 英文的wiki寫的更詳細： https://en.wikipedia.org/wiki/Modbus
- 一般的分類：
    - Modbus/RTU: 在RS-485, RS-232這類的串列通訊上使用
    - Modbus/TCP: 基於TCP的Modbus協定，因此只要是走網路的大致上都可以適用
    - 另外還有modbus ascii，這類似於Modbus/RTU，但通訊資料量較大
- 一個Master vs 多個slave
- 一問一答的通訊方式
- 若對通訊格式有興趣可以參考：
    - https://www.rtaautomation.com/technologies/modbus-tcpip/


## Demo
- mb_demo0: (初學可跳過)，在不知道Modbus設備的通訊參數下可以使用此demo來掃出何種參數才是正確的
- mb_demo1: 配合FATEK PLC進行通訊控制，在此demo中將對PLC的DO進行控制：OFF-->ON-->OFF-->ON-->OFF
- mb_demo2: 配合FATEK PLC進行通訊控制，在此demo中將會讀回PLC的DI，當接上的磁簧開關是ON時會讀回1；磁簧開關斷開時會讀回0
- mb_demo3: 配合電表，讀回輸入的交流電電壓

### 開始前需要先安裝的套件
- pip install serial
- pip install modbus_tk
    - modbus_tk 支援 Modbus/TCP, Modbus/RTU
    - 其中Modbus/RTU需要serial的套件，因此要自行另外安裝

### Demo1 : PLC DO控制

In [2]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time


In [3]:
mbComPort = 'COM7'   # your RS-485 port. for linux --> "/dev/ttyUSB2"
baudrate = 9600
databit = 8 #7, 8
parity = 'N' #N, O, E
stopbit = 1 #1, 2
mbTimeout = 100 # ms


In [5]:

mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 1
addr = 2 #base0 --> my 110V Led燈泡

for i in range(5):
    try:
        value = i%2
        #-- FC5: write multi-coils
        rr = master.execute(mbId, cst.WRITE_SINGLE_COIL, addr, output_value=value)
        print("Write(addr, value)=",  rr)

    except Exception as e:
        print("modbus test Error: " + str(e))

    time.sleep(2)

master._do_close()


Write(addr, value)= (2, 0)
Write(addr, value)= (2, 65280)
Write(addr, value)= (2, 0)
Write(addr, value)= (2, 65280)
Write(addr, value)= (2, 0)


True

### other example for reading DI, DO, AI, AO (補充教材)

In [ ]:
## other example for reading DI, DO, AI, AO

addr = 1
n = 4

#-- DI read: FC2  Read multi-input discrete ( 1xxxx )
rr = master.execute(mbId, cst.READ_DISCRETE_INPUTS, addr, n)
print("DI value= ", rr)

#-- FC01: Read multi-coils status (0xxxx) for DO
rr = master.execute(mbId, cst.READ_COILS, addr, n)
print("DO value= ",  rr)

#-- FC04: read multi-input registers (3xxxx), for AI
rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, n)
print("AI value= ", rr)

#-- FC03: read multi-registers (4xxxx) for AO
rr = master.execute(mbId, cst.READ_HOLDING_REGISTERS, addr, n)
print("AO value= ", rr)



### Demo2: PLC DI點讀取
- 範例中，第二個DI是ON，因此讀回來為1

In [6]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time

In [10]:
mbComPort = 'COM7'
baudrate = 9600
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

In [12]:
mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 1
addr = 1000 #base0

for i in range(5):
    try:
        # FATEK的PLC把DI點放在DO的address中
        #-- FC01: Read multi-coils status (0xxxx) for DO
        rr = master.execute(mbId, cst.READ_COILS, addr, 4)
        print("value= ",  rr)

    except Exception as e:
        print("modbus test Error: " + str(e))

    time.sleep(1)

master._do_close()


value=  (0, 1, 0, 0)
value=  (0, 1, 0, 0)
value=  (0, 1, 0, 0)
value=  (0, 1, 0, 0)
value=  (0, 1, 0, 0)


True

### Demo3: 電表電壓資訊讀取

In [13]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu
import time
from struct import *

In [19]:
mbComPort = 'COM7' #your RS-485 port
baudrate = 19200
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

In [39]:
mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
master = modbus_rtu.RtuMaster(mb_port)
master.set_timeout(mbTimeout/1000.0)

mbId = 4
#[0x1000-0x1001]=VIn_a
addr = 0x1000#4096

try:
    # FC3
    rr = master.execute(mbId, cst.READ_INPUT_REGISTERS, addr, 4)
    print('read value:', rr)

    # convert to float:
    # IEEE754 ==> (Hi word Hi Bite, Hi word Lo Byte, Lo word Hi Byte, Lo word Lo Byte)
    try:
        v_a_hi = rr[1]
        v_a_lo = rr[0]
        v_a = unpack('>f', pack('>HH', v_a_hi, v_a_lo))
        print('v_a=', v_a)
        #struct.unpack(">f",'\x42\xd8\x6b\x8d')
    except Exception as e:
        print(e)

except Exception as e:
    print("modbus test Error: " + str(e))


master._do_close()


read value: (27533, 17112, 0, 0)
v_a= (108.2100601196289,)


True

### 補充資料
### 掃Modbus設備 --> 用於不知道設備的baudrate等參數時

- 以下例的結果來說，代表這19200 8N1這個參數下是有回應的，代表可以如此通訊：

scan @ 19200 N 1
rr: (2,)

In [41]:
import serial
import modbus_tk
import modbus_tk.defines as cst
import modbus_tk.modbus_rtu as modbus_rtu


mbComPort = 'COM7'
baudrate = 38400
databit = 8
parity = 'N'
stopbit = 1
mbTimeout = 100 # ms

# scan_test:
for baudrate in [9600, 19200, 38400, 115200]:
    for parity in ['N', 'O', 'E']:
        for stopbit in[1, 2]:
            print('scan @', baudrate, parity, stopbit)
            mb_port = serial.Serial(port=mbComPort, baudrate=baudrate, bytesize=databit, parity=parity, stopbits=stopbit)
            master = modbus_rtu.RtuMaster(mb_port)
            master.set_timeout(mbTimeout/1000.0)

            mbId = 1
            addr = 1

            try:
                # FC3
                rr = master.execute(mbId, cst.READ_HOLDING_REGISTERS, addr, 1)
                print('rr:', rr)

            except Exception as e:
                print("modbus test Error: " + str(e))

            master._do_close()


scan @ 9600 N 1
modbus test Error: Response length is invalid 0
scan @ 9600 N 2
modbus test Error: Response length is invalid 0
scan @ 9600 O 1
modbus test Error: Response length is invalid 0
scan @ 9600 O 2
modbus test Error: Response length is invalid 0
scan @ 9600 E 1
modbus test Error: Response length is invalid 0
scan @ 9600 E 2
modbus test Error: Response length is invalid 0
scan @ 19200 N 1
rr: (2,)
scan @ 19200 N 2
rr: (2,)
scan @ 19200 O 1
modbus test Error: Invalid CRC in response
scan @ 19200 O 2
modbus test Error: Invalid CRC in response
scan @ 19200 E 1
modbus test Error: Invalid CRC in response
scan @ 19200 E 2
modbus test Error: Invalid CRC in response
scan @ 38400 N 1
modbus test Error: Response length is invalid 0
scan @ 38400 N 2
modbus test Error: Response length is invalid 0
scan @ 38400 O 1
modbus test Error: Response length is invalid 0
scan @ 38400 O 2
modbus test Error: Response length is invalid 0
scan @ 38400 E 1
modbus test Error: Response length is invalid 0